In [1]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd

# Running To-Do List for Scraper
- Implement logging
- Add something that will pull character images from their profile and save them in a folder for later use (By.XPATH, "//img[@class='charmanimage']").get_attribute("src")

In [2]:
cs = CompendiumScraper('\\Users\\jasre\\Code\\ws_config.yml')

In [3]:
cs.generate_ability_dict('jackgarland', JP=True)

In [4]:
cs.parse_ability_dict_to_df('jackgarland', JP=True)

,char_name,ability_name,main_target_hp_attacks,non_target_hp_attacks,hp_dmg_cap_up_perc,attribute_list
0,jackgarland,BRV Attack+,0,0,0,[Melee]
1,jackgarland,BRV Attack++,0,0,0,[Melee]
2,jackgarland,HP Attack+,1,0,0,[Melee]
3,jackgarland,HP Attack++,1,0,0,[Melee]
4,jackgarland,Spinning Slash,4,4,20,"[Group, Melee]"
5,jackgarland,Soul Burst (Uncapped HP Attack),1,0,900,"[Melee, FollowUp]"
6,jackgarland,Soul Burst,1,0,0,"[Melee, FollowUp]"
7,jackgarland,Highbringer,8,1,20,[Melee]
8,jackgarland,Attack Boost Up,0,0,0,"[Instant, NoSummon, NoEX]"
9,jackgarland,Lightbringer,6,6,10,"[Group, Melee]"
